In [220]:
import pandas as pd
import numpy as np

In [221]:
train_df = pd.read_csv('dataset/train.csv')
train_df.shape

(201917, 6)

In [222]:
test_df = pd.read_csv('dataset/test.csv')
test_df.shape

(123623, 5)

In [223]:
submit_df = pd.read_csv('dataset/sample_submission.csv')

In [224]:
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [225]:
test_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [226]:
from scipy.stats import mode
df = pd.concat([train_df, test_df], sort=False)
df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [227]:
df.first_active_month = df.first_active_month.fillna(mode(df.first_active_month).mode[0])

/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [228]:
df['is_train'] = df['target'].notnull() 
df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,is_train
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,True
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,True
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,True
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,True
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,True


In [229]:
df['first_active_month'] = df['first_active_month'].apply(pd.to_datetime)
df.sample(4)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,is_train
12509,2017-10-01,C_ID_18dd52d189,1,1,0,NaN,False
80035,2016-10-01,C_ID_7181793610,2,2,0,NaN,False
9713,2016-06-01,C_ID_74c562ad4d,3,1,1,NaN,False
106703,2017-09-01,C_ID_16c9981872,3,2,1,NaN,False


In [230]:
print(df.card_id.nunique() / df.shape[0])

1.0


In [231]:
df['first_active_mo'] = df['first_active_month'].dt.month
df['first_active_yr'] = df['first_active_month'].dt.year

In [232]:
cat_columns = ['feature_1', 'feature_2', 'feature_3', 'first_active_mo', 'first_active_yr']
df = pd.get_dummies(df, columns=cat_columns)

In [233]:
train_df = df.loc[df['is_train'] == True].set_index('card_id')
test_df = df.loc[df['is_train'] == False].set_index('card_id')

y_train = train_df.pop('target').values
train_df = train_df.drop(['first_active_month', 'is_train'], axis=1)
X_train = train_df.loc[:,:].values

test_df = test_df.drop(['first_active_month', 'is_train', 'target'], axis=1)
X_test = test_df.loc[:,:].values

In [234]:
print(X_train.shape)
print(X_test.shape)

(201917, 30)
(123623, 30)


In [239]:
import xgboost
xgb = xgboost.XGBRegressor(n_estimators=106)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=106,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [240]:
y_pred = regressor.predict(X_test)
y_pred

array([-0.62219751, -0.71831984, -0.15331648, ..., -0.95196138,
       -0.4515594 , -0.48294357])

In [241]:
sample_file = pd.read_csv('dataset/sample_submission.csv')
sample_file['target'] = y_pred
sample_file.to_csv('elo_submission1.csv', index=False)